In [ ]:
############################################  WARNING   #########################################################
# This notebook can not be run currently. The model is not fine-tuned and the implementation for DataPipeline has changed.
#################################################################################################################

In [1]:
import os
import shutil
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # not enough memory in GPU
from KAGGLE_NLP_with_disaster_tweets.model.simple_bert_model import SimpleBertModel
from KAGGLE_NLP_with_disaster_tweets.data_preparation.utils import DataPipeline, BatchPipeline
import tensorflow_text  # required for BERT model

log_dir = 'logs/hyperparameter_tuning_bertmodel'
try:
    # clearing logging directory
    shutil.rmtree(log_dir)
except NotADirectoryError:
    pass

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [2]:
train_file_name = 'train.csv'
test_file_name = 'test.csv'
sample_submission_file_name = 'sample_submission.csv'

In [3]:
# BERT model has its own preprocessor, so we don't use our own
data_pipeline = DataPipeline(train_file_name, test_file_name, sample_submission_file_name, vectorize=False, quiet=True)

2022-01-19 23:44:07.866220: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-19 23:44:07.866270: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sersery-asusVivo
2022-01-19 23:44:07.866276: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sersery-asusVivo
2022-01-19 23:44:07.866419: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-01-19 23:44:07.866442: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-01-19 23:44:07.866447: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-01-19 23:44:07.866689: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU 

In [4]:
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf

parameters = {
    "embedding_dim": 64,
    "n_labels": 2,
    "epochs": 3,
}

hyperparameters = {"learning_rate": hp.HParam("learning_rate", hp.RealInterval(1e-3, 1e-1)),
                   "hidden_unit": hp.HParam("hidden_unit", hp.Discrete([8, 16, 32, 64, 128])),
                   "batch_size": hp.HParam("batch_size", hp.Discrete([16, 32, 64])),
                   "optimizer": hp.HParam("optimizer", hp.Discrete(["adam", "adamw", "sgd"])),
                   "class_weights": hp.HParam("class_weights", hp.Discrete(["none", "balanced"])),
                   "dropout": hp.HParam("dropout", hp.Discrete(0.1, 0.2, 0.3))}

In [5]:
from sklearn.utils import class_weight
targets = data_pipeline.dataframe["target"]
balanced_class_weights = dict(enumerate(class_weight.compute_class_weight("balanced",
                                                                          classes=targets.unique().tolist(),
                                                                          y=targets.tolist())))

In [6]:
log_directory = "./logs/hyperparameter_tuning_bertmodel/"
with tf.summary.create_file_writer(log_directory).as_default():
    hp.hparams_config(
        hparams=hyperparameters.values(),
        metrics=[hp.Metric("accuracy", display_name="Accuracy"),
                 hp.Metric("precision", display_name="Precision"),
                 hp.Metric("recall", display_name="Recall"),
                 hp.Metric("f1", display_name="F1 Score")])

In [7]:
batch_size = 32
parameters["epochs"] = 3

hparams = {
    hyperparameters["optimizer"]: "adamw",
    hyperparameters["hidden_unit"]: 16,
    hyperparameters["batch_size"]: batch_size,
    hyperparameters["learning_rate"]: 1e-5,
    hyperparameters["class_weights"]: "balanced",
    hyperparameters["dropout"]: 0.1
}

bert_model = SimpleBertModel(batch_pipeline=BatchPipeline(data_pipeline.dataset, batch_size),
                             parameters=parameters,
                             hyperparameters=hyperparameters,
                             hparams=hparams,
                             class_weights=balanced_class_weights)

accuracy, precision, recall, f1 = bert_model.fit_and_evaluate(log_directory=log_directory)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score: {f1}")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocess (KerasLayer)        {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

2022-01-19 23:45:01.610163: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 937635840 exceeds 10% of free system memory.


190/190 [==============================] - 49s 239ms/step - loss: 5.2833 - accuracy: 0.5479 - val_loss: 3.0970 - val_accuracy: 0.5978
Epoch 2/3
190/190 [==============================] - ETA: 0s - loss: 4.1006 - accuracy: 0.5888

2022-01-19 23:45:44.910063: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 937635840 exceeds 10% of free system memory.


190/190 [==============================] - 43s 228ms/step - loss: 4.1006 - accuracy: 0.5888 - val_loss: 3.0030 - val_accuracy: 0.6033
Epoch 3/3
190/190 [==============================] - ETA: 0s - loss: 3.9113 - accuracy: 0.6016

2022-01-19 23:46:27.844372: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 937635840 exceeds 10% of free system memory.


23/23 [==============================] - 2s 69ms/step - loss: 3.0603 - accuracy: 0.6073
run=0__lr=1e-05__hidden_unit=16__batch_size=32__optimizer=adamw__class_weights=balanced__dropout=0.3 completed.
Accuracy: 0.6073369383811951
Precision: 0.6846160619936598
Recall: 0.5627023470387236
F1-Score: 0.5000082272604398


In [8]:
%load_ext tensorboard

In [9]:
tensorboard --logdir logs/hyperparameter_tuning_bertmodel --port 5001

Launching TensorBoard...